## TPRU: Tax Model Demonstration
#### August 23, 2019

The [Policy Simulation Library](https://www.pslmodels.org) (PSL) hosts open-source, policy-related models that conform to certain standards for model I/O, documentation, and testing.

In this notebook, we illustrate how three of these models ([`Tax-Calculator`](https://github.com/PSLmodels/Tax-Calculator), [`Behavioral-Responses`](https://github.com/PSLmodels/Behavioral-Responses), [`Cost-of-Capital-Calculator`](https://github.com/PSLmodels/Cost-of-Capital-Calculator), [`OG-USA`](https://github.com/PSLmodels/OG-USA)) can work together to provide a broad analysis of set of tax policies.


#### Setup
Before running this notebook, you must open a terminal window and install the Tax-Calculator (taxcalc), Cost-of-Capital-Calculator (ccc), and OG-USA (ogusa) packages.
* [`Tax-Calculator`](https://github.com/PSLmodels/Tax-Calculator) (taxcalc): This package can be downloaded from Anaconda. Go to terminal and type `conda install -c PSLmodels taxcalc`
* [`Behavioral-Responses`](https://github.com/PSLmodels/Behavioral-Responses) (behresp): This package can be downloaded from Anaconda. Go to terminal and type `conda install -c PSLmodels behresp`
* [`Cost-of-Capital-Calculator`](https://github.com/PSLmodels/Cost-of-Capital-Calculator) (ccc): This package can be downloaded from Anaconda. Go to terminal and type `conda install -c PSLmodels ccc`.
* [`OG-USA`](https://github.com/PSLmodels/OG-USA) (ogusa): This package must be cloned and set up a specific environment. Follow the instructions in the main [`README.md`](https://github.com/PSLmodels/OG-USA/blob/master/README.md).

In [1]:
# imports
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
# import PSL models
import taxcalc
import ccc
import behresp
from ogusa import output_plots as op
# to print bokeh plots inline
output_notebook()
# to print matplotlib plots inline
%matplotlib inline

Loading BokehJS ...

## Revenue estimates and distributional analysis of the personal income tax with Tax-Calculator

We'll start by using the `Tax-Calculator` model, a microsimulation model of individual income taxes and payroll taxes.  This model can be used for revenue estimation, distributional analysis, effective tax rate calculations, and other uses.

As an example to work with, we'll simulate the Tax Cuts and Jobs Act.  Since this represents current law, which is the baseline policy in Tax-Calculator, we'll need to specify changes to this that represent tax law in 2017:

In [2]:
law2017_url = ('https://raw.githubusercontent.com/'
              'PSLmodels/Tax-Calculator/master/taxcalc/'
              'reforms/2017_law.json')
law2017_dict = taxcalc.Policy.read_json_reform(law2017_url)
law2017_dict

{'CPI_offset': {2017: 0.0025},
 'II_rt1': {2018: 0.1},
 'II_brk1': {2017: [9325, 18650, 9325, 13350, 18650]},
 'II_rt2': {2018: 0.15},
 'II_brk2': {2017: [37950, 75900, 37950, 50800, 75900]},
 'II_rt3': {2018: 0.25},
 'II_brk3': {2017: [91900, 153100, 76550, 131200, 153100]},
 'II_rt4': {2018: 0.28},
 'II_brk4': {2017: [191650, 233350, 116675, 212500, 233350]},
 'II_rt5': {2018: 0.33},
 'II_brk5': {2017: [416700, 416700, 208350, 416700, 416700]},
 'II_rt6': {2018: 0.35},
 'II_brk6': {2017: [418400, 470700, 235350, 444550, 470700]},
 'II_rt7': {2018: 0.396},
 'PT_rt1': {2018: 0.1},
 'PT_brk1': {2017: [9325, 18650, 9325, 13350, 18650]},
 'PT_rt2': {2018: 0.15},
 'PT_brk2': {2017: [37950, 75900, 37950, 50800, 75900]},
 'PT_rt3': {2018: 0.25},
 'PT_brk3': {2017: [91900, 153100, 76550, 131200, 153100]},
 'PT_rt4': {2018: 0.28},
 'PT_brk4': {2017: [191650, 233350, 116675, 212500, 233350]},
 'PT_rt5': {2018: 0.33},
 'PT_brk5': {2017: [416700, 416700, 208350, 416700, 416700]},
 'PT_rt6': {2018

In [3]:
# Create calculators for 2017 law and TCJA
pol = taxcalc.policy.Policy()
rec = taxcalc.records.Records.cps_constructor()
tc_tcja = taxcalc.calculator.Calculator(policy=pol, records=rec)
pol.implement_reform(law2017_dict)
tc_2017 = taxcalc.calculator.Calculator(policy=pol, records=rec)

CTC_c was redefined in release 1.0.0



### Revenue Estimation

In [4]:
# Compute annual revenue for each year in a 10 year budget window
# Do this for 2017 law and the TCJA
rev_2017 = {'iit': {}, 'payroll': {}}
rev_tcja = {'iit': {}, 'payroll': {}}
for t in range(2018, 2028):
    tc_2017.advance_to_year(t)
    tc_2017.calc_all()
    tc_tcja.advance_to_year(t)
    tc_tcja.calc_all()
    rev_2017['iit'][t] = tc_2017.weighted_total('iitax')
    rev_2017['payroll'][t] = tc_2017.weighted_total('payrolltax')
    rev_tcja['iit'][t] = tc_tcja.weighted_total('iitax')
    rev_tcja['payroll'][t] = tc_tcja.weighted_total('payrolltax')

In [5]:
# make table of revenue estimates
rev_2017_df = pd.DataFrame.from_dict(rev_2017).T
rev_tcja_df = pd.DataFrame.from_dict(rev_tcja).T
diff_df = (rev_tcja_df - rev_2017_df) * 1e-9
pd.options.display.float_format = '${:.3f}'.format
diff_df.sum(axis=1)

iit       $-1264.258
payroll       $0.000
dtype: float64

### Distributional Analysis and Effective Rates Calculations

In [6]:
# Restart calculation to get back to first year
pol = taxcalc.policy.Policy()
tc_tcja = taxcalc.calculator.Calculator(policy=pol, records=rec)
pol.implement_reform(law2017_dict)
tc_2017 = taxcalc.calculator.Calculator(policy=pol, records=rec)
tc_2017.advance_to_year(2018)
tc_tcja.advance_to_year(2018)
tc_2017.calc_all()
tc_tcja.calc_all()
# Average tax rate plot
atr_plot = tc_2017.atr_graph(tc_tcja)
show(atr_plot)

CTC_c was redefined in release 1.0.0



In [7]:
pch_plot = tc_2017.pch_graph(tc_tcja)
show(pch_plot)

In [8]:
# Distributional table
tc_2017.difference_table(tc_tcja, 'standard_income_bins', 'combined')

,count,tax_cut,perc_cut,tax_inc,perc_inc,mean,tot_change,share_of_change,ubi,benefit_cost_total,benefit_value_total,pc_aftertaxinc
<$0K,$0.050,$0.000,$0.000,$0.000,$0.100,$108.500,$0.005,$-0.000,$0.000,$0.000,$0.000,$0.000
=$0K,$0.910,$0.000,$0.000,$0.000,$0.000,$0.000,$0.000,$-0.000,$0.000,$0.000,$0.000,$nan
$0-10K,$6.150,$0.180,$2.900,$0.330,$5.300,$-2.100,$-0.013,$0.000,$0.000,$0.000,$0.000,$0.000
$10-20K,$8.990,$4.570,$50.800,$1.330,$14.800,$-71.300,$-0.641,$0.400,$0.000,$0.000,$0.000,$0.500
$20-30K,$17.360,$9.710,$56.000,$1.890,$10.900,$-124.400,$-2.160,$1.400,$0.000,$0.000,$0.000,$0.500
$30-40K,$20.250,$11.100,$54.800,$1.910,$9.400,$-244.900,$-4.959,$3.300,$0.000,$0.000,$0.000,$0.800
$40-50K,$16.430,$10.180,$62.000,$1.820,$11.100,$-357.300,$-5.871,$3.900,$0.000,$0.000,$0.000,$0.900
$50-75K,$30.250,$20.380,$67.400,$3.460,$11.400,$-490.000,$-14.822,$9.800,$0.000,$0.000,$0.000,$0.900
$75-100K,$19.590,$14.820,$75.600,$2.220,$11.300,$-730.100,$-14.303,$9.500,$0.000,$0.000,$0.000,$1.000
$100-200K,$30.170,$25.170,$83.400,$3.750,$12.400,$-1348.600,$-40.685,$27.000,$0.000,$0.000,$0.000,$1.200


In [9]:
# Marginal tax rate plot
mtr_plot = tc_2017.mtr_graph(tc_tcja)
show(mtr_plot)

## Including behavioral responses

Using the `Behavior-Response` package, we can apply partial-equilibrium behavioral responses to the analysis using `Tax-Calculator`:

In [10]:
# specify assumed non-zero response-function substitution elasticity
response_elasticities = {'sub': 0.6}
# apply a behavioral response to the change in after-tax income from the reform
_, df2br = behresp.response(tc_2017, tc_tcja, response_elasticities)

# Calculate reform income tax liabilities for 2018 with behavioral response
itax_rev2br = (df2br['iitax'] * df2br['s006']).sum()
# Calculate baseline income tax liabilities for 2018
base_rev = tc_2017.weighted_total('iitax')
# Calculate reform income tax liabilities for 2018 without behavior responses
reform_rev_static = tc_tcja.weighted_total('iitax')

# Change in total income tax revenue estimates for 2018 (policy-baseline)
# (estimates in billons of dollars)
print('{}_STATIC_itax_rev_change($B)= {:.3f}'.format(
    2018, (reform_rev_static - base_rev) * 1e-9))
print('{}_DYNAMIC_itax_rev_change($B)= {:.3f}'.format(
    2018, (itax_rev2br - base_rev) * 1e-9))

2018_STATIC_itax_rev_change($B)= -150.727
2018_DYNAMIC_itax_rev_change($B)= -88.040


## Effective tax rates with the Cost-of-Capital-Calculator

Another important consideration in tax policy are the incentive effects of taxation on business investment.  The Cost-of-Capital-Calculator (CCC) is a model that allows one to explore effects of tax policy on the cost of capital and effective tax rates (ETR).

CCC computes the the cost of capital CCC and ETRS across business entity tax treatment (e.g., corporate vs non-corporate), indusry, and asset type.  When considering the effects of the individual income tax on savings and investment decisions, CCC uses the output from Tax-Calculator.

Here, we use CCC to provide an analysis of the effects of the TCJA on investment incentives.

In [11]:
# Load asset data
assets = ccc.data.Assets()
assets.df.head(n=5)

,Unnamed: 0,tax_treat,assets,bea_asset_code,bea_ind_code,Industry,minor_code_alt,major_asset_group,minor_asset_group,major_industry,GDS Class Life,GDS Life,ADS Life,System,Method,asset_name,delta
0,0,corporate,$0.000,RD32,110C,Farms,$111.000,Intellectual Property,Intellectual Property,"Agriculture, forestry, fishing, and hunting",$7.000,$7.000,$10.000,GDS,Expensing,Aerospace products and parts manufacturing,$0.220
1,1,corporate,$0.000,RD32,113F,"Forestry, fishing, and related activities",$113.000,Intellectual Property,Intellectual Property,"Agriculture, forestry, fishing, and hunting",$7.000,$7.000,$10.000,GDS,Expensing,Aerospace products and parts manufacturing,$0.220
2,2,corporate,$0.000,RD32,113F,"Forestry, fishing, and related activities",$114.000,Intellectual Property,Intellectual Property,"Agriculture, forestry, fishing, and hunting",$7.000,$7.000,$10.000,GDS,Expensing,Aerospace products and parts manufacturing,$0.220
3,3,corporate,$0.000,RD32,2110,Oil and gas extraction,$211110.000,Intellectual Property,Intellectual Property,Mining,$7.000,$7.000,$10.000,GDS,Expensing,Aerospace products and parts manufacturing,$0.220
4,4,corporate,$0.000,RD32,2120,"Mining, except oil and gas",$212110.000,Intellectual Property,Intellectual Property,Mining,$7.000,$7.000,$10.000,GDS,Expensing,Aerospace products and parts manufacturing,$0.220


In [12]:
# To compare two policies, create two model specifications
# Then use the Calculator class to compute ETRs (and more)
# across the two specifications
# Create an instance of the Specification class
spec_2017 = ccc.parameters.Specification(year=2018, call_tc=True, iit_reform=law2017_dict)
business_law_2017 = business_tax_reform = {
    'CIT_rate': 0.35, 'BonusDeprec_3yr': 0.50, 'BonusDeprec_5yr': 0.50,
    'BonusDeprec_7yr': 0.50, 'BonusDeprec_10yr': 0.50,
    'BonusDeprec_15yr': 0.50, 'BonusDeprec_20yr': 0.50}
spec_2017.update_specification(business_law_2017)
calc_2017 = ccc.calculator.Calculator(spec_2017, assets)
spec_tcja = ccc.parameters.Specification(year=2018, call_tc=True, iit_reform={})
calc_tcja = ccc.calculator.Calculator(spec_tcja, assets)

CTC_c was redefined in release 1.0.0

year:  2018
{'tau_nc': array([0.25110782]), 'tau_div': array([0.19778403]), 'tau_int': array([0.34412168]), 'tau_scg': array([0.33487396]), 'tau_lcg': array([0.20103027]), 'tau_td': array([0.25219687]), 'tau_h': array([0.16588951])}
year:  2018
{'tau_nc': array([0.20079347]), 'tau_div': array([0.1895823]), 'tau_int': array([0.31289917]), 'tau_scg': array([0.29351868]), 'tau_lcg': array([0.19022541]), 'tau_td': array([0.21890162]), 'tau_h': array([0.04378408])}


In [13]:
# Summary table
pd.options.display.float_format = '{:.2f}%'.format
calc_2017.summary_table(
    calc_tcja, output_variable='metr',
    include_land=True, include_inventories=True)

,,Marginal Effective Tax Rate Under Baseline Policy,Marginal Effective Tax Rate Under Reform Policy,Change from Baseline (pp)
0,Overall,10.04%,2.96%,-7.08%
1,Corporations,13.32%,3.73%,-9.59%
2,Equity Financed,25.41%,11.85%,-13.56%
3,Debt Financed,-56.55%,-29.57%,26.98%
4,Pass-Through Entities,23.49%,20.42%,-3.06%
5,Equity Financed,22.51%,17.76%,-4.76%
6,Debt Financed,28.13%,31.50%,3.37%


In [14]:
# Plot to show variation in METRs across assets
rplot = calc_2017.range_plot(
    calc_tcja, corporate=True, output_variable='metr',
    include_land=False, include_inventories=False)
show(rplot)

In [15]:
# Visualizing changes by asset
aplot = calc_2017.grouped_bar(calc_tcja, include_land=True, include_inventories=True)
show(aplot)

In [16]:
# Visualizing changes by industry
iplot = calc_2017.grouped_bar(calc_tcja, group_by_asset=False)
show(iplot)

In [ ]:
bplot = calc_2017.asset_bubble(calc_tcja)
show(bplot)

## Macro results with OG-USA

OG-USA is a overlapping generations model used to evaluate fiscal policy.  OG-USA is calibrated to the U.S. economy, but we've run this source code with a calibration to Italy (Calling the model OG-ITA) and are in the process of calibrating OG-India.

OG-USA will show aggregate effects of tax policy (as well as impacts across generations and skill groups) from the current period until the economy reaches its long-run steady state.

OG-USA takes some time to run (1+ hours to solve the model), so we've cached some results where we simulated the transition from 2017 law to the TCJA.  

In [17]:
# Load in OG-USA results and parameterizations
base_tpi = pickle.load(open('./TCJA_Results/OUTPUT_BASELINE/partial/TPI/TPI_vars.pkl', 'rb'))
base_params = pickle.load(open('./TCJA_Results/OUTPUT_BASELINE/partial/model_params.pkl', 'rb'))
reform_tpi = pickle.load(open('./TCJA_Results/OUTPUT_REFORM/partial/TPI/TPI_vars.pkl', 'rb'))
reform_params = pickle.load(open('./TCJA_Results/OUTPUT_REFORM/partial/model_params.pkl', 'rb'))

Exception ignored in: <_io.FileIO name='./TCJA_Results/OUTPUT_BASELINE/partial/TPI/TPI_vars.pkl' mode='rb' closefd=True>
Exception ignored in: <_io.FileIO name='./TCJA_Results/OUTPUT_BASELINE/partial/model_params.pkl' mode='rb' closefd=True>
Exception ignored in: <_io.FileIO name='./TCJA_Results/OUTPUT_REFORM/partial/TPI/TPI_vars.pkl' mode='rb' closefd=True>
Exception ignored in: <_io.FileIO name='./TCJA_Results/OUTPUT_REFORM/partial/model_params.pkl' mode='rb' closefd=True>


In [ ]:
# Plot OG-USA output; interest rates
rplot = op.plot_aggregates(
    base_tpi, base_params, reform_tpi=reform_tpi,
    reform_params=reform_params, var_list=['r_gov'],
    plot_type='levels', num_years_to_plot=22,
    vertical_line_years=[
        base_params.start_year + base_params.tG1,
        base_params.start_year + base_params.tG2],
    plot_title='Real Interest Rates Under Baseline and Reform')

In [ ]:
# Plot OG-USA output; government debt
dplot = op.plot_gdp_ratio(base_tpi, base_params, reform_tpi, reform_params,
                   var_list=['D'], num_years_to_plot=22,
                   start_year=2019, vertical_line_years=[
                           base_params.start_year + base_params.tG1,
                           base_params.start_year + base_params.tG2],
                   plot_title='Debt-to-GDP')

In [ ]:
# Plot OG-USA output; percentage changes in macro vars (Y, K, L, C)
pct_chg_plot = op.plot_aggregates(base_tpi, base_params, reform_tpi=reform_tpi,
                    reform_params=reform_params,
                    var_list=['Y', 'K', 'L', 'C'], plot_type='pct_diff',
                    num_years_to_plot=22,
                    vertical_line_years=[
                        base_params.start_year + base_params.tG1,
                        base_params.start_year + base_params.tG2],
                    plot_title='Percentage Changes in Macro Aggregates')